In [47]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [53]:
import torch 
import torch.autograd as autograd 
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

In [27]:
def get_data():
    !wget https://archive.ics.uci.edu/ml/machine-learning-databases/00275/Bike-Sharing-Dataset.zip
    !unzip Bike-Sharing-Dataset.zip
    !mv Readme.txt *.csv data

In [28]:
get_data()

--2021-04-14 16:30:02--  https://archive.ics.uci.edu/ml/machine-learning-databases/00275/Bike-Sharing-Dataset.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 279992 (273K) [application/x-httpd-php]
Saving to: ‘Bike-Sharing-Dataset.zip’

Bike-Sharing-Datase 100%[===================>] 273.43K  --.-KB/s    in 0.1s    

2021-04-14 16:30:02 (2.49 MB/s) - ‘Bike-Sharing-Dataset.zip’ saved [279992/279992]

Archive:  Bike-Sharing-Dataset.zip
  inflating: Readme.txt              
  inflating: day.csv                 
  inflating: hour.csv                


In [29]:
day = pd.read_csv("data/day.csv")

In [30]:
day.head()

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,331,654,985
1,2,2011-01-02,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,131,670,801
2,3,2011-01-03,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,120,1229,1349
3,4,2011-01-04,1,0,1,0,2,1,1,0.200000,0.212122,0.590435,0.160296,108,1454,1562
4,5,2011-01-05,1,0,1,0,3,1,1,0.226957,0.229270,0.436957,0.186900,82,1518,1600


## Make a two-layer neural network that predicts `cnt`

1. Write a model that uses just continuous variables.<br>
    Remember that variables need to be normalized.
2. One-hot encode all categorical variables. Train a model with both numerical and categorical variables.
3. Finished everything? Try using embedding to encode the categorical variables.
4. Print training loss, validation loss and r^2

In [31]:
day["day"] = pd.to_datetime(day.dteday)

In [34]:
thr_day = day.day.quantile(.8)
thr_day

Timestamp('2012-08-07 00:00:00')

In [35]:
train = day[day["day"] < thr_day].copy()
val = day[day["day"] >= thr_day].copy()

In [36]:
train.shape, val.shape

((584, 17), (147, 17))

In [37]:
train.columns

Index(['instant', 'dteday', 'season', 'yr', 'mnth', 'holiday', 'weekday',
       'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed',
       'casual', 'registered', 'cnt', 'day'],
      dtype='object')

In [42]:
cols = train.columns[[4,5,6,7,9,10,11,12,12,13,14]]
cols

Index(['mnth', 'holiday', 'weekday', 'workingday', 'temp', 'atemp', 'hum',
       'windspeed', 'windspeed', 'casual', 'registered'],
      dtype='object')

In [63]:
x_train = train.loc[:,cols].values
y_train = train.cnt.values

In [64]:
x_val = val.loc[:,cols].values
y_val = val.cnt.values

In [65]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_val = scaler.transform(x_val)

In [66]:
x_train.shape

(584, 11)

In [67]:
model = nn.Sequential(
    nn.Linear(11, 10),
    nn.ReLU(),
    nn.Linear(10, 1)
)

In [68]:
x_train = torch.tensor(x_train).float()
y_train = torch.tensor(y_train).float()

x_val = torch.tensor(x_val).float()
y_val = torch.tensor(y_val).float()

In [69]:
learning_rate = 0.1
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [77]:
def train_model(model, optimizer, epochs=10000):
    for t in range(epochs):
    # Forward pass: compute predicted y using operations on Variables
       model.train()
       y_hat = model(x_train)
       loss = F.mse_loss(y_hat, y_train.unsqueeze(1))
       
       # Before the backward pass, use the optimizer object to zero all of the
       # gradients for the variables
       optimizer.zero_grad()
       loss.backward()
    
       # Calling the step function on an Optimizer makes an update to its
       # parameters
       optimizer.step()
       model.eval()
       y_hat_val = model(x_val)
       val_loss = F.mse_loss(y_hat_val, y_val.unsqueeze(1))
       if t % 1000 == 0: print("train loss %.3f valid loss %.3f" % (loss.item(), val_loss.item()))

In [99]:
model = nn.Sequential(
    nn.Linear(11, 20),
    nn.ReLU(),
    nn.Linear(20, 1)
)

In [100]:
learning_rate = 1
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [101]:
train_model(model, optimizer, epochs=10000)

train loss 20448158.000 valid loss 37880200.000
train loss 231.362 valid loss 6549.518
train loss 28.536 valid loss 4851.934
train loss 134.963 valid loss 5106.808
train loss 242.401 valid loss 4657.684
train loss 21.997 valid loss 5166.168
train loss 2.133 valid loss 3780.667
train loss 6170.809 valid loss 10872.777
train loss 4.729 valid loss 2650.672
train loss 0.367 valid loss 2249.938


In [102]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.3)
train_model(model, optimizer, epochs=10000)

train loss 6.641 valid loss 22366.326
train loss 2.650 valid loss 1859.121
train loss 0.007 valid loss 1825.167
train loss 0.006 valid loss 1792.404
train loss 1.178 valid loss 1761.766
train loss 0.663 valid loss 1720.490
train loss 0.338 valid loss 1676.377
train loss 11.072 valid loss 1727.499
train loss 199.110 valid loss 1709.656
train loss 2.867 valid loss 1660.197


In [103]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
train_model(model, optimizer, epochs=10000)

train loss 386.705 valid loss 4795.107
train loss 0.193 valid loss 1623.643
train loss 0.002 valid loss 1623.043
train loss 0.019 valid loss 1621.648
train loss 0.002 valid loss 1623.795
train loss 0.758 valid loss 1628.195
train loss 0.001 valid loss 1620.028
train loss 0.002 valid loss 1616.911
train loss 0.286 valid loss 1607.066
train loss 0.394 valid loss 1617.729
